We can't reasonably expect to understand what an article is saying based on what words are in the headline alone. It simply doesn't give enough information. And there's no standardized way to do it without having x thousand inputs for every single type of word there is. We need to perform sentiment analysis FIRST, and then from there we can feed it into the model. 

So first we want a way to get the article fast and painlessly. The ideal method would be to run this fully through scrapy to pull out important data, but before we do that it's important to use a developmental level approach before we move on to a full scale scraper. 

### Update 1/5/2021:
With pr_fullpull mostly implemented and working, attention is now turned to what we actually do with the article. We're going to start by pulling an individual article and go from there. 

There will be an ONLINE and OFFLINE modes, and sentiment will be one of the ONLINE modes. We do not want to store the entire article text manually, just use it temporarily. 
ON: datetime, sentiment
OFF: 

In [46]:
#textarray
timestamp_date

'Feb 9, 2021 6:00AM EST'

In [27]:
sentence.replace('\n','')

' NANAIMO, British Columbia--(BUSINESS WIRE)--Tilray Inc. (NASDAQ: TLRY), a global pioneer in cannabis research, cultivation, production, and distribution, today announced it has established an agreement with Grow Pharma to import and distribute Tilray’s medical cannabis products into the United Kingdom (UK). This agreement allows Tilray to provide authorized UK patients in need with a locally maintained supply of medical cannabis solutions.'

In [44]:
import pandas as pd
import numpy as np
from selenium import webdriver #import
from selenium.webdriver.support import expected_conditions
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from bs4 import BeautifulSoup

# pull an article from the archives
#filedata=pd.read_csv('newsreleases_071421.csv', names=['tick',])

ticker='TLRY'
# example url
url = 'https://www.nasdaq.com/press-release/tilrayr-announces-agreement-with-grow-pharma-to-import-and-distribute-medical'
#url = 'https://www.nasdaq.com/press-release/addus-homecare-signs-definitive-agreement-to-acquire-queen-city-hospice-llc-and'
# pull with selenium for now, then translate into scrapy
driver=webdriver.Edge("C:/Users/cukel/Downloads/edgedriver_win32/msedgedriver.exe")
driver.get(url)
# alternative using beautiful soup
soupage=BeautifulSoup(driver.page_source,'lxml')
# get elements with soup
# should work around stale references?
driver.close()
header=soupage.find('h1', class_='press-release-header__title').text.replace('\n','').replace('  ','')
timestamp_date=soupage.find('time', class_='timestamp__date').text
content=soupage.find_all('p')

fulltext=''
textarray=[]
# pull out the actual article
for paragraph in content:
    # need to remove initial location/time/report stuff -> later
    text=paragraph.get_text()
    text=text.replace('\n','').replace('  ','')
    # stop condition is if the first word is about
    # and the sentence/paragraph is under a few words
    # let's assume under 6 words as a lenient rule
    if text.split()[0].lower()=='about' and len(text.split())<7:
        break
    textarray.append(text)
    fulltext=fulltext+text+' '

# NEED to format for nltk usage
# export needs to be in encapsulating code
# need to move it over to scrapy
# try to make it, check things if it fails


In [64]:
# get ticker articles
ticker='ACRS'
pd.read_csv("C:/Users/cukel/OneDrive/Documents/PyScripts/hotshot finance type shit/ticker_releasedata/raw/{0}.txt".format(ticker),names=['link'])['link'].tolist()

['https://www.nasdaq.com/press-release/aclaris-therapeutics-to-participate-in-the-h.c.-wainwright-bioconnect-virtual',
 'https://www.nasdaq.com/press-release/aclaris-therapeutics-announces-publication-of-preclinical-research-of-zunsemetinib-in',
 'https://www.nasdaq.com/press-release/aclaris-therapeutics-to-host-virtual-rd-day-on-december-7-2021-2021-11-18',
 'https://www.nasdaq.com/press-release/aclaris-therapeutics-to-participate-in-upcoming-investor-conferences-2021-11-03',
 'https://www.nasdaq.com/press-release/aclaris-therapeutics-reports-third-quarter-2021-financial-results-and-provides-a',
 'https://www.nasdaq.com/press-release/aclaris-therapeutics-announces-proposed-settlement-and-settlement-hearing-of',
 'https://www.nasdaq.com/press-release/aclaris-therapeutics-to-participate-in-upcoming-investor-conferences-2021-08-26',
 'https://www.nasdaq.com/press-release/aclaris-therapeutics-reports-second-quarter-2021-financial-results-and-provides-a',
 'https://www.nasdaq.com/press-rel

In [4]:
# download new ntlk packages
#nltk.download()
nltk.__file__

'C:\\Users\\cukel\\anaconda3\\lib\\site-packages\\nltk\\__init__.py'

In [65]:
# now that we have text, we analyze it
# lean heavily on natrual language toolkit
from nltk import tokenize
from nltk.sentiment import SentimentAnalyzer
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from nltk.sentiment.util import *

# Instantiate the sentiment intensity analyzer with the existing lexicon
#vader = SentimentIntensityAnalyzer()

# tokenize an individual sentence
# MAY THROW ERRORS LATER when above code changes
# split into sentences

#test it out first
#sentence='bette white has always been popular and has been good in recent films'

sid=SentimentIntensityAnalyzer()
ss=sid.polarity_scores(fulltext)
for k in sorted(ss):
    print('{0}: {1}, '.format(k,ss[k]),end='')

compound: 0.986, neg: 0.005, neu: 0.882, pos: 0.113, 

{'neg': 0.005, 'neu': 0.882, 'pos': 0.113, 'compound': 0.986}

In [ ]:

timedata=pd.read_csv('newsreleases_071421_FULLTIME.csv')
df = pd.merge(filedata, timedata, on='tick')
df.drop(labels='link_y', axis=1, inplace=True)
df.rename(columns={'link_x':'link'}, inplace=True)


In [47]:
from dateutil import parser
def getdatetime(dtime):
    return parser.parse(dtime,tzinfos={"EST":-5*3600, "EDT":-4*3600})

In [48]:
getdatetime(timestamp_date)

datetime.datetime(2021, 2, 9, 6, 0, tzinfo=tzoffset('EST', -18000))